In [3]:
import pandas as pd
df = pd.read_csv('./ChatbotData.csv', encoding='utf-8')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [1]:
#setence 모듈설치
%pip install sentence_transformers

     ---------------------------------------- 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 5.8/5.8 MB 19.6 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 18.1 MB/s eta 0:00:00
     ---------------------------------------- 182.4/182.4 kB ? eta 0:00:00
     ---------------------------------------- 151.7/151.7 kB ? eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 26.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=55c778f78aa9a0a26db91ccb17c148938c735056fcef8ff5b145c94fb80eca89
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\62\f2\10\1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import random
import numpy as np

#지수로그 없애기 위해 소수점 6자리까지만
np.set_printoptions(precision=6, suppress=True)

In [5]:
#한국어 모델 불러오기
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [6]:
#임베딩 잘되는지 테스트
sentence = ['안녕하세요 테스트입니다.','반갑습니다. 테스트입니다.']
embeding = model.encode(sentence)
embeding.shape

(2, 768)

In [6]:
#자연어 챗봇 데이터 불러오기 
!wget https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [7]:
df = pd.read_csv('ChatbotData.csv')
df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [8]:
#매번 채팅 들어올때 임베딩하면 시간이 너무 오래 걸려 챗봇 질문 데이터 임베딩해서 csv로 저장하기
import csv
with open('embeding.csv','w') as file :
    mapdata = []
    write = csv.writer(file)
    for temp in df['Q']:
        t = model.encode(temp)
        mapdata.append( t )
        
    write.writerows(mapdata)

In [9]:
#csv 불러오기
df1 = pd.read_csv('embeding.csv',header=None)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Columns: 768 entries, 0 to 767
dtypes: float64(768)
memory usage: 69.3 MB


In [10]:
#챗봇 데이터 불러오기
df = pd.read_csv('ChatbotData.csv')
df.iloc[0]

Q            12시 땡!
A        하루가 또 가네요.
label             0
Name: 0, dtype: object

In [11]:
#챗봇 함수 입력받은 텍스트와 embeding 유사도 체크후 원본 챗봇에 cos컬럼 만든후 상위 정렬
def chatbot_text(text):
    em_result = model.encode(text)
    co_result = []
    for temp in range(len(df1)):
        data = df1.iloc[temp]
        co_result.append( cosine_similarity([em_result],[data])[0][0] )
    df['cos'] = co_result
    df_result = df.sort_values('cos',ascending=False)
    r = random.randint(0,5)
    return df_result.iloc[r]

chatbot_text("배고파요")

Q               배고파
A        뭐 좀 챙겨드세요.
label             0
cos        0.911879
Name: 2001, dtype: object

In [12]:
import time
start = time.time() 
chatbot_text("배고파요")
print("time :", time.time() - start)

time : 2.3280725479125977


In [13]:
chatbot_text("배고파요")

Q        밥 먹었는데 배고파
A          디져트 드세요.
label             0
cos        0.930304
Name: 1964, dtype: object

In [14]:
chatbot_text("서울시 관악구 삼성동")

Q        서울 근교에서 갈만한 곳 추천
A             드라이브 추천해들요.
label                   0
cos              0.976281
Name: 2428, dtype: object

### ======================================

In [17]:
import pandas as pd
df = pd.read_csv('./test.CSV', encoding='euc-kr')
df.head()

,시,구,동,의료기관명,주소,전화번호
0,서울,강남구,삼성동,강남구보건소,서울 강남구 삼성동(삼성2동) 8 강남구보건소,02-3423-5555
1,서울,강남구,일원동,삼성서울병원,서울 강남구 일원로81 삼성서울병원,02-1599-3114
2,서울,강남구,도곡동,강남세브란스병원,서울 강남구 언주로211 강남세브란스병원,02-1599-6114
3,서울,강남구,도곡동,강남베드로병원,남부순환로2637,02-1544-7522
4,서울,강동구,성내동,강동구보건소,서울 강동구 성내로45,02-3425-6713


In [18]:
df['Q'] = df['시'] + " " + df['구'] + " " + df['동']
df['A'] = df['의료기관명'] + " " + df['주소'] + " " + df['전화번호']

In [19]:
df.head()

,시,구,동,의료기관명,주소,전화번호,Q,A
0,서울,강남구,삼성동,강남구보건소,서울 강남구 삼성동(삼성2동) 8 강남구보건소,02-3423-5555,서울 강남구 삼성동,강남구보건소 서울 강남구 삼성동(삼성2동) 8 강남구보건소 02-3423-5555
1,서울,강남구,일원동,삼성서울병원,서울 강남구 일원로81 삼성서울병원,02-1599-3114,서울 강남구 일원동,삼성서울병원 서울 강남구 일원로81 삼성서울병원 02-1599-3114
2,서울,강남구,도곡동,강남세브란스병원,서울 강남구 언주로211 강남세브란스병원,02-1599-6114,서울 강남구 도곡동,강남세브란스병원 서울 강남구 언주로211 강남세브란스병원 02-1599-6114
3,서울,강남구,도곡동,강남베드로병원,남부순환로2637,02-1544-7522,서울 강남구 도곡동,강남베드로병원 남부순환로2637 02-1544-7522
4,서울,강동구,성내동,강동구보건소,서울 강동구 성내로45,02-3425-6713,서울 강동구 성내동,강동구보건소 서울 강동구 성내로45 02-3425-6713


In [22]:
df = df.drop(columns=['시','구','동','의료기관명','주소','전화번호'])

In [23]:
df.head()

,Q,A
0,서울 강남구 삼성동,강남구보건소 서울 강남구 삼성동(삼성2동) 8 강남구보건소 02-3423-5555
1,서울 강남구 일원동,삼성서울병원 서울 강남구 일원로81 삼성서울병원 02-1599-3114
2,서울 강남구 도곡동,강남세브란스병원 서울 강남구 언주로211 강남세브란스병원 02-1599-6114
3,서울 강남구 도곡동,강남베드로병원 남부순환로2637 02-1544-7522
4,서울 강동구 성내동,강동구보건소 서울 강동구 성내로45 02-3425-6713


In [24]:
df.to_csv('test1.csv', index=False)

UnicodeDecodeError: 'euc_kr' codec can't decode byte 0xec in position 5: illegal multibyte sequence

In [25]:
df = pd.read_csv('./test1.csv')
df.head()

,Q,A
0,서울 강남구 삼성동,강남구보건소 서울 강남구 삼성동(삼성2동) 8 강남구보건소 02-3423-5555
1,서울 강남구 일원동,삼성서울병원 서울 강남구 일원로81 삼성서울병원 02-1599-3114
2,서울 강남구 도곡동,강남세브란스병원 서울 강남구 언주로211 강남세브란스병원 02-1599-6114
3,서울 강남구 도곡동,강남베드로병원 남부순환로2637 02-1544-7522
4,서울 강동구 성내동,강동구보건소 서울 강동구 성내로45 02-3425-6713


In [26]:
df = pd.read_csv('test1.csv')

import csv
with open('embeding1.csv','w') as file :
    mapdata = []
    write = csv.writer(file)
    for temp in df['Q']:
        t = model.encode(temp)
        mapdata.append( t )
        
    write.writerows(mapdata)

In [39]:
df1 = pd.read_csv('embeding1.csv',header=None)
df = pd.read_csv('test1.csv')

def chatbot_text(text):
    em_result = model.encode(text)
    co_result = []
    for temp in range(len(df1)):
        data = df1.iloc[temp]
        co_result.append( cosine_similarity([em_result],[data])[0][0] )
    df['cos'] = co_result
    df_result = df.sort_values('cos',ascending=False)
    return df_result.iloc[0:5]

In [40]:
chatbot_text("서울 강남구 삼성동")

,Q,A,cos
0,서울 강남구 삼성동,강남구보건소 서울 강남구 삼성동(삼성2동) 8 강남구보건소 02-3423-5555,1.000000
1,서울 강남구 일원동,삼성서울병원 서울 강남구 일원로81 삼성서울병원 02-1599-3114,0.994959
2,서울 강남구 도곡동,강남세브란스병원 서울 강남구 언주로211 강남세브란스병원 02-1599-6114,0.993015
3,서울 강남구 도곡동,강남베드로병원 남부순환로2637 02-1544-7522,0.993015
5,서울 강동구 둔촌동,중앙보훈병원 서울 강동구 진황도로 61길 53 02-2225-1114,0.991159


In [28]:
df.head()

,Q,A,cos
0,서울 강남구 삼성동,강남구보건소 서울 강남구 삼성동(삼성2동) 8 강남구보건소 02-3423-5555,0.994289
1,서울 강남구 일원동,삼성서울병원 서울 강남구 일원로81 삼성서울병원 02-1599-3114,0.988198
2,서울 강남구 도곡동,강남세브란스병원 서울 강남구 언주로211 강남세브란스병원 02-1599-6114,0.986865
3,서울 강남구 도곡동,강남베드로병원 남부순환로2637 02-1544-7522,0.986865
4,서울 강동구 성내동,강동구보건소 서울 강동구 성내로45 02-3425-6713,0.966298


In [37]:
chatbot_text("서울 강남구 삼성")

Q                                         서울 강남구 삼성동
A      강남구보건소 서울 강남구 삼성동(삼성2동) 8 강남구보건소 02-3423-5555
cos                                         0.987866
Name: 0, dtype: object

In [38]:
chatbot_text("서울 강남 삼성")

Q                                    서울 광진구 화양동
A      건국대학교병원 서울특별시 광진구 능동로 120-1 02-1588-1533
cos                                    0.978327
Name: 13, dtype: object

## =================================

In [41]:
df = pd.read_csv('./호흡기환자진료센터_질문답변.csv')
df.head()

,Unnamed: 0,Unnamed: 1,질문,답변
0,진료센터,0,서울 강남구 삼성동,"병원명은 (의)미래의료재단리드림의원 주소는 서울특별시 강남구 테헤란로 513, (삼..."
1,진료센터,1,서울 강남구 대치동,"병원명은 365열린가정의학과의원 주소는 서울특별시 강남구 도곡로 405, 삼환 아르..."
2,진료센터,2,서울 강남구 논현동,"병원명은 365코이이비인후과의원 주소는 서울특별시 강남구 강남대로118길 12, 부..."
3,진료센터,3,서울 강남구 삼성동,"병원명은 가온경희한의원 주소는 서울특별시 강남구 선릉로 654, 예인빌딩 401호 ..."
4,진료센터,4,서울 강남구 삼성동,"병원명은 강남365가정의학과의원 주소는 서울특별시 강남구 학동로 346, 강남구청역..."


In [42]:
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 1'])

In [43]:
df.head()

,질문,답변
0,서울 강남구 삼성동,"병원명은 (의)미래의료재단리드림의원 주소는 서울특별시 강남구 테헤란로 513, (삼..."
1,서울 강남구 대치동,"병원명은 365열린가정의학과의원 주소는 서울특별시 강남구 도곡로 405, 삼환 아르..."
2,서울 강남구 논현동,"병원명은 365코이이비인후과의원 주소는 서울특별시 강남구 강남대로118길 12, 부..."
3,서울 강남구 삼성동,"병원명은 가온경희한의원 주소는 서울특별시 강남구 선릉로 654, 예인빌딩 401호 ..."
4,서울 강남구 삼성동,"병원명은 강남365가정의학과의원 주소는 서울특별시 강남구 학동로 346, 강남구청역..."


In [44]:
df.columns = ['Q','A']

In [45]:
df.head()

,Q,A
0,서울 강남구 삼성동,"병원명은 (의)미래의료재단리드림의원 주소는 서울특별시 강남구 테헤란로 513, (삼..."
1,서울 강남구 대치동,"병원명은 365열린가정의학과의원 주소는 서울특별시 강남구 도곡로 405, 삼환 아르..."
2,서울 강남구 논현동,"병원명은 365코이이비인후과의원 주소는 서울특별시 강남구 강남대로118길 12, 부..."
3,서울 강남구 삼성동,"병원명은 가온경희한의원 주소는 서울특별시 강남구 선릉로 654, 예인빌딩 401호 ..."
4,서울 강남구 삼성동,"병원명은 강남365가정의학과의원 주소는 서울특별시 강남구 학동로 346, 강남구청역..."


In [46]:
df.to_csv('호흡기환자진료센터_질문답변1.csv')

In [47]:
df = pd.read_csv('./호흡기환자진료센터_질문답변1.csv')

import csv
with open('embeding_호흡기.csv','w') as file :
    mapdata = []
    write = csv.writer(file)
    for temp in df['Q']:
        t = model.encode(temp)
        mapdata.append( t )
        
    write.writerows(mapdata)

In [48]:
df1 = pd.read_csv('embeding_호흡기.csv',header=None)
df = pd.read_csv('호흡기환자진료센터_질문답변1.csv')

def chatbot_text(text):
    em_result = model.encode(text)
    co_result = []
    for temp in range(len(df1)):
        data = df1.iloc[temp]
        co_result.append( cosine_similarity([em_result],[data])[0][0] )
    df['cos'] = co_result
    df_result = df.sort_values('cos',ascending=False)
    return df_result.iloc[0:5]

In [49]:
chatbot_text("서울 강남구 삼성동")

,Unnamed: 0,Q,A,cos
0,0,서울 강남구 삼성동,"병원명은 (의)미래의료재단리드림의원 주소는 서울특별시 강남구 테헤란로 513, (삼...",1.0
220,220,서울 강남구 삼성동,"병원명은 코모키이비인후과의원 주소는 서울특별시 강남구 테헤란로87길 13, 서영빌딩...",1.0
49,49,서울 강남구 삼성동,"병원명은 나가정의학과의원 주소는 서울특별시 강남구 삼성로 651, 501호 (삼성동...",1.0
53,53,서울 강남구 삼성동,"병원명은 다소니의원 주소는 서울특별시 강남구 봉은사로73길 5, 3층 (삼성동) 전...",1.0
214,214,서울 강남구 삼성동,"병원명은 청담이든소아청소년과의원 주소는 서울특별시 강남구 삼성로 620, 블래스톤리...",1.0


In [50]:
chatbot_text("서울 강남구 대치동")

,Unnamed: 0,Q,A,cos
168,168,서울 강남구 대치동,"병원명은 연세키즈소아청소년과의원 주소는 서울특별시 강남구 남부순환로 2927, 대치...",1.0
25,25,서울 강남구 대치동,"병원명은 강창순내과의원 주소는 서울특별시 강남구 남부순환로 2941, 201호 (대...",1.0
27,27,서울 강남구 대치동,"병원명은 고시환의원 주소는 서울특별시 강남구 남부순환로 2936, 304호 (대치동...",1.0
174,174,서울 강남구 대치동,"병원명은 우리들내과의원 주소는 서울특별시 강남구 도곡로63길 12, 204,205호...",1.0
31,31,서울 강남구 대치동,"병원명은 굿닥터이비인후과의원 주소는 서울특별시 강남구 도곡로 449, 2층 202호...",1.0


In [51]:
chatbot_text("서울 강남구 다치동")

,Unnamed: 0,Q,A,cos
137,137,서울 강남구 대치동,"병원명은 안효승내과의원 주소는 서울특별시 강남구 선릉로 406, 4층 (대치동) 전...",0.998147
59,59,서울 강남구 대치동,"병원명은 대치서울영재소아청소년과의원 주소는 서울특별시 강남구 도곡로 515, 2층 ...",0.998147
27,27,서울 강남구 대치동,"병원명은 고시환의원 주소는 서울특별시 강남구 남부순환로 2936, 304호 (대치동...",0.998147
31,31,서울 강남구 대치동,"병원명은 굿닥터이비인후과의원 주소는 서울특별시 강남구 도곡로 449, 2층 202호...",0.998147
158,158,서울 강남구 대치동,"병원명은 연세신앤송이비인후과의원 주소는 서울특별시 강남구 테헤란로78길 8, 7층 ...",0.998147


In [59]:
chatbot_text("서울 영등포구 당산제1동")

,Unnamed: 0,Q,A,cos
2445,2445,서울 영등포구 당산동,"병원명은 삼성의원 주소는 서울특별시 영등포구 당산로36길 12, 201동 204호 ...",0.992939
2516,2516,서울 영등포구 당산동,병원명은 의료법인 인봉의료재단 영등포병원 주소는 서울특별시 영등포구 당산로31길 1...,0.992939
2415,2415,서울 영등포구 당산동,"병원명은 로뎀나무내과의원 주소는 서울특별시 영등포구 영신로 259, 1층 (당산동5...",0.992939
2420,2420,서울 영등포구 당산동,"병원명은 맑은숲한의원 주소는 서울특별시 영등포구 당산로 223, 5~6층 (당산동5...",0.992939
2439,2439,서울 영등포구 당산동,"병원명은 베토벤이비인후과의원 주소는 서울특별시 영등포구 양평로 22, 3층 (당산동...",0.992939
